バウンディングボックス
* クラスと物体を含む矩形のこと。
* まずバウンディングボックスを予測する必要がある。

ローカライゼーション
* 物体の位置を特定する事

物体検知output
1. 入力画像から固定サイズのウィンドウをすべての可能な位置で取得する
2. これらのパッチ(領域)を画像分類器に入力し、分類問題としてモデル化する。

ダウンサンプリング
1. 元の画像を複数サイズにサイズ変更した画像を用意する。
1. これらの画像のいずれかで、選択したサイズのウィンドウ内に物体が完全に含められる。
1. 一般的には、特定の条件(典型的には最小サイズに達する)が得られるまで、画像のダウンサンプリング(サイズが縮小)を行う。
1. これらの画像のそれぞれについて、 固定サイズのウィンドウ検出を実行する。
1. ピラミッド型の構造になる。
1. このようなピラミッドには64レベルまでのレベルがあるのが一般的。
1. これらのウィンドウは全て、関心のあるオブジェクトを検出するために 分類器に入力され、サイズと場所の問題を解決します。

アスペクト比
1. もう1つ、縦横比(アスペクト比、aspect ratio)問題がある。
1. 座っている人が立っている人や眠っている人とは異なるアスペクト比を持つように、多く􏰃物体が さまざまな形で存在する。

**領域提案**（オブジェクトが含まれている可能性のあるイメージ内の領域）を検出する。

->Edge Boxes[1] などのアルゴリズムを使用して領域提案を生成する。


領域提案から CNN 特徴量を抽出。

->提案領域はイメージからトリミングされ、サイズ変更される。

->その後、トリミングされてサイズ変更された領域は、CNN によって分類される。

抽出した特徴量を使用してオブジェクトを分類する。

->最後に、CNN 特徴量を使用して学習したサポート ベクター マシン (SVM) によって、領域提案境界ボックスが調整される。


### MEMO
RPNは小さなニューラルネットワークで、特徴マップ上をn×nサイズのスライディングwindowで走査し、各々のn×nサイズの注目領域をネットワークの入力とする。そして、各スライディングwindow位置に対してk個の候補領域を推定する。

k個の候補領域を推定するために、RPNは以下2つの全結合層へ分岐する。

cls layer：物体かどうか(objectness)を分類する
reg layer：Bounding Boxの回帰を行う

cls layerには、k個の各候補領域がオブジェクトか、背景かの確率を推定した2k個のスコアが出力される。
reg layerには、k個のBounding Boxの座標・サイズを表す4k個の出力がある。(x座標、y座標、幅、高さ)

この論文ではn=3とし、実際にはスライディングwindowではなく3×3サイズの畳み込み層と、それに続く2つに分岐した1×1サイズの畳み込み層(Fully Convolutional Networks)として実装している。
(これは3×3のスライディングwindowと等価)


「特徴マップを抽出するCNNをRPNも共有している」RPN自体も畳み込みニューラルネットワークの構造を持っている




Region Propsal Network(RPN)はCNNの特徴マップ（つまりResnet等のCNN層出力）を入力。

ある領域が物体か背景か(objectness)およびアンカーの位置の補正データ(corrdinates)を出力する。
objectnessは0-1の値で1に近いほど物体である確証が高い。
corrdinatesはBindingBoxの四角の座標について補正する量を出力する。

RPNは一箇所につきk個のBindingBoxを提案する。
これは一つの物体について重複することが多く、最も確度の高い高いBindingBoxのみ残したい（refining)。大体k=128などが使われることがある通り、refineなしでは大量のpredicitionが生成されてしまい実用的でない。

Refineに使われる概念がAnchorである。
物体検出において物体は3x3のような正方形であることは少ない。

例えば人間はだいたい縦長であったり、車は横長で有ることが多い。このような図形情報を扱うために提案された概念がAnchorである。
座標に付き提案するBindingboxに対しAnchor情報とどれくらい異なるかを導出し、一番Anchorに近いBindingBoxをMain Predictionとして出力する。これによって物体一つにつきBindingBoxを一つに絞り込み、かつ一番高精度なものを残すことができる。


Fast R-CNN に Region Proposal Network を導入して、更に高速化したアルゴリズムCNN の畳み込み層を使ってFeature Map を生成する
Sliding Window で切り出したFeature Map から領域候補を生成
論文では Feature Map の各点に対して、９種類のサイズの Anchor Box を定義している



## R-CNN 
物体検出􏰃タスクに対しても CNNのアルゴリズムを上手く応用できないか?という課題を解く先駆けとなった論文

物体検出はより正確な畳み込みニューラルネットワークに基づく分類器に置き換えられまたが、 CNNは非常に遅く、 計算上コストが高く、スライディングウィンドウ検出器によって生成された非常に多くのパッチで CNNを実行することは不可能だった問題を**Selective Search**と呼ばれる物体候補 (object proposal)アルゴリズムを使用することで解決した。

画像を取り込み、画像内の主要な物体を**bounding box**を介して、正確に特定する。


### アルゴリズム
1. 画像を入力
1. **Selective Search**を用いて**領域候補 (Region Proposals)**を2000個程度切り出す。
   * Selective Searchでは複数のスケールのウィンドウを調べ、テクスチャ、色、または強度を共有する隣接ピクセルを探し物体を識別します。 
   * 画像の中からたくさんのボックスの候補をリストアップし (**region proposal**)、 それらをどれかが実際に物体に対応しているかどうかを調べる。
1. 領域候補の領域画像を 全て一定の大きさの正方形にリサイズして CNNにかけて特徴量を取り出す
1. 1. 取り出した特徴量を使って複数のSVMによって学習し分類を推定する。
   1. 取り出した特徴量を使って回帰によりバウンディングボックスの正確な位置を推定する。
1. 出力:画像内の各物体の境界ボックス +ラベル


### 欠点

* 学習を各目的ごとに別々に学習する必要がある
  * CNNのFine-tune
  * 複数のSVMによるクラス分類 (Classification)
  * 物体􏰃詳細位置推定 (Bounding Box Regression)
* 実行時間がすごく遅い :GPUを使って10-45 s/image

![](https://jp.mathworks.com/help/vision/ug/rcnn_ja_JP.png)



## SPP-net(spatial pyramid pooling)

### 従来手法の問題
Selective Search によって生成された2000の領域候補(region proposal)上でCNNを実行すると、 多くの時間がかかる。


### 問題の改善
その領域に対応する**最後の畳み込み層の特徴マップのその部分だけにプーリング操作を実行する**ことで実現。

画像1枚から一回のCNNで大きな特徴マップを作成した後、 領域候補の特徴を SPPによってベクトル化し、スピードは GPU上にて24-102倍に高速化を実現。 (2000個の領域候補はかなり領域の重複が多いため、重複する画像領域を CNNで特徴抽出するのはかなり無駄)

中間層で起こるダウンサンプリング( VGGの場合の座標を単に 16で割る)を考慮に入れて、 畳み込み層上の領域を投影することによって、領域に対応する畳み込み層の矩形部分の計算可能。





### Fast R-CNN 
* 領域提案をイメージ全体で処理します。
* 各領域提案に対応する CNN 特徴量をプーリングし分類する。
* オーバーラップする領域の計算を共有するので、R-CNN よりも効率的。
* 関数 trainFastRCNNObjectDetector を使用して Fast R-CNN オブジェクト検出器を学習させます。
* この関数は、イメージからオブジェクトを検出する fastRCNNObjectDetector を返します。
* 実はRegion Proposal の生成に時間がかかる
![](https://jp.mathworks.com/help/vision/ug/fast_ja_JP.png)

### Faster R-CNN
* Edge Boxes などの外部アルゴリズムを使用する代わりに、領域提案ネットワーク (RPN) を追加してネットワーク内で直接領域提案を生成する。
* 途中までの演算を共有化(Sliding Window で切り出したFeature Map から領域候補を生成)することで追加の演算コストを最小化する
* アンカーボックスによるオブジェクトの検出を使用する。
* ネットワーク内で領域提案を生成する方が高速であり、データに合わせてより適切に調整されます。
* 関数 trainFasterRCNNObjectDetector を使用して Faster R-CNN オブジェクト検出器を学習させます。
* この関数は、イメージからオブジェクトを検出する fasterRCNNObjectDetector を返します。
![](https://jp.mathworks.com/help/vision/ug/faster_ja_JP.png)


In [ ]:
画像を重なり合うタイルに分割する
上記のスライディングウィンドウと同様に、イメージをより小さく、
隣接するタイル同士が少しずつ重なりあうように分割(オーバーラップタイル)します。ここでは77個の小さな画像にしています。

### MEMO

Faster R-CNNはRegionProposalもCNN化することで物体検出モデルを全てDNN化し、高速化するのがモチベーションとなっている。

途中までの演算を共有化(Sliding Window で切り出したFeature Map から領域候補を生成)することで追加の演算コストを最小化する

またFaster-RCNNはMulti-task lossという学習技術を使っており、RegionProposalモデルも込でモデル全体をend-to-endで学習させることに成功している。

Faster-RCNNはCNN出力（特徴マップ）を元にregion proposal(物体があるっぽい領域を抽出）するモデルを構築している。

実際のregion proposalは3~4層ほどのCNNで構成可能で小さい。

Faster R-CNNでは、従来Selective Searchで行っていた候補領域(region proposals)の検出処理をRPN (Region Proposal Network)というニューラルネットワークに置き換えてさらなる効率化を実現した。

convolution layer = detect 詳細な形状の出力
pooling layer = detect 大雑把な形を出力

### attention
* Attentionは文章のような前後の並びが重要なデータを上手く扱う事ができる
* 並列処理の実現や遠く離れた単語間の関連性を捉える事も可能
* Transformerは画像、オーディオ、ビデオなどにも応用していく予定

RNN(Recurrent Neural Networks)やLSTM(Long Short-Term Memory networks)やgated RNNsは機械翻訳や自然言語のモデリングなど、シーケンスな処理、つまり前後の繋がりに意味があるため順番で連続して処理する必要がある処理に良く使われます。

RNNやCNNは、単語単位で文頭から順番に処理をしていきます。この順番に処理をする事が原因で、処理を並列化する事が難しくなります。さらに、文章が非常に長い場合、ニューラルネットワークは、以前の内容を忘れてしまったり、または他の位置の単語の内容と混合してしまう傾向があります。

最近の研究では、因数分解や条件演算を使って計算効率を向上し、モデル性能の大幅な改善も達成されています。しかし、これらはシーケンスな処理の基本的な制約、つまり平行処理が難しい事を解決するには十分ではありません。

Attentionメカニズムは、ニューラルネットワークが忘却をしてしまう問題を克服する解決策の1つです。Attentionメカニズムは入力シーケンスや出力シーケンス内の距離に関わらず文脈の依存関係をモデリング可能なのです。この特徴により、Attentionはシーケンシャル処理や伝達モデルに不可欠になっています。しかし、ほとんどの場合、RNNと連携してAttentionメカニズムは使用されます。

# 論文読解

以下の論文を読み問題に答えてください。CNNを使った物体検出（Object Detection）の代表的な研究です。


[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99


https://arxiv.org/pdf/1506.01497.pdf

# 問題
それぞれについてJupyter Notebookにマークダウン形式で記述してください。

### 条件
* 答える際は論文のどの部分からそれが分かるかを書く。
* 必要に応じて先行研究（引用されている論文）も探しにいく。最低2つは他の論文を利用して回答すること。
* 論文の紹介記事を見ても良い。ただし、答えは論文内に根拠を探すこと。

## (1) 物体検出の分野にはどういった手法が存在したか。


* SPPnet
* Fast R-CNN

*Advances like SPPnet [1] and Fast R-CNN [2] have reduced the running time of these detection networks, - Abstract*

[1] *K. He, X. Zhang, S. Ren, and J. Sun,“Spatial pyramid pooling in deep convolutional networks for visual recognition,” in European Conference on Computer Vision (ECCV), 2014.*

[2] *R. Girshick, “Fast R-CNN,” in IEEE International Conference on Computer Vision (ICCV), 2015.*

## (2) Fasterとあるが、どういった仕組みで高速化したのか。


物体領域提案ネットワーク（RPN）の導入
* 検出ネットワークと全画像の畳み込み特徴量を共有しボトルネックであった領域計算のコストをほぼゼロにした。

*In this work, we introduce a Region Proposal Network (RPN) that shares full-image
convolutional features with the detection network, thus enabling nearly cost-free region proposals. - Abstract*

## (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。


One-Stage
* 物体領域検出とクラス分類を同時に行う。

Two-Stage
* 第1段階で物体領域検出を行う。
* 第2段階でクラス分類を行う。

*In OverFeat, the region-wise features come from a sliding window of one aspect ratio over a scale pyramid. These features are used to simultaneously determine the location and category of objects. In RPN, the features are from square (3×3) sliding windows and predict proposals relative to anchors with different scales and aspect ratios. Though both methods use sliding windows, the region proposal task is only the first stage of Faster RCNN—the downstream Fast R-CNN detector attends to the proposals to refine them. In the second stage of our cascade, the region-wise features are adaptively pooled [1], [2] from proposal boxes that more faithfully cover the features of the regions. - One-Stage Detection vs. Two-Stage Proposal + Detection.*

*P. Sermanet, D. Eigen, X. Zhang, M. Mathieu, R. Fergus, and Y. LeCun, “Overfeat: Integrated recognition, localization and detection using convolutional networks,” in International Conference on Learning Representations (ICLR), 2014.*

## (4) RPNとは何か。

RPN (Region Proposal Network)は、物体領域候補の検出を行う。画像全体からCNNで抽出した特徴マップ（full-image convolutional features）に対して、候補領域のBounding Boxと、その領域の物体らしさ(Objectness)を表すスコアを出力する。
* Fast R-CNN に Region Proposal Network を導入して、更に高速化したアルゴリズム
* CNN の畳み込み層で生成されたFeature Mapを転用し、Sliding Window で畳み込みを行ったFeature Mapから領域提案を生成する。

*An RPN is a fully convolutional
network that simultaneously predicts object bounds and objectness scores at each position. - Abstract*

## (5) RoIプーリングとは何か。


畳み込み処理を行ったfeature mapから、region proposalにあたる部分領域をうまく「固定サイズのfeature map」として抽出する。

*The RoI pooling layer [2] in Fast R-CNN accepts the convolutional features and also the predicted bounding boxes as input, so a theoretically valid backpropagation solver should also involve gradients w.r.t. the box coordinates. These gradients are ignored in the above approximate joint training. In a non-approximate joint training solution, we need an RoI pooling layer that is differentiable w.r.t. the box coordinates.*
*- 3.2 Sharing Features for RPN and Fast R-CNN*

## (6) Anchorのサイズはどうするのが適切か。


Anchorは特徴マップ上の各点となる。

*An anchor is centered at the sliding window in question, and is associated with a scale and aspect ratio (Figure 3, left).  - Multi-Scale Anchors as Regression References. - 3.1.1 Anchors*

## (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

* PASCAL VOC
* mAP

*We primarily evaluate detection mean Average Precision (mAP), because this is the actual metric for object detection (rather than focusing on object proposal proxy metrics). -  4.1 Experiments on PASCAL VOC*